In [1]:
import torch
import glob
import os
import pathlib
import cv2
import shutil

import numpy                as np
import pandas               as pd
import matplotlib.pyplot    as plt
import cv2                  as cv 
import matplotlib.pyplot    as plt
import torch.nn.functional  as F
import seaborn              as sns
import plotly.express       as px
import plotly.graph_objects as go

from random                  import sample
from plotly.subplots         import make_subplots
from google.colab            import drive
from google.colab.patches    import cv2_imshow
from skimage                 import io
from PIL                     import Image
from torch                   import nn, optim
from torchvision             import datasets, transforms
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print("Versão:",torch.__version__)

Versão: 1.10.0+cu111


In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
dir_path_train = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/80_20_Brain_Tumor/BrainTumor_80'
dir_path_test = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/80_20_Brain_Tumor/BrainTumor_20'

In [4]:
transform_file = transforms.Compose(
    [
     transforms.Resize([128,128]),
     transforms.ToTensor()
    ]
)

In [5]:
train_dataset = datasets.ImageFolder(dir_path_train, transform=transform_file)
test_dataset = datasets.ImageFolder(dir_path_test, transform=transform_file)

### Pre-processamento de dados

In [6]:
def get_class_distribution(dataset_obj):
  count_dict = {k:0 for k, v in dataset_obj.class_to_idx.items()}
  for i in dataset_obj:
    y_labels = i[1]
    y_labels = idx2class[y_labels]
    count_dict[y_labels] += 1
  return count_dict

In [7]:
idx2class = {v: k for k, v in train_dataset.class_to_idx.items()}
df_treino = pd.DataFrame.from_dict([get_class_distribution(train_dataset)]).melt()

idx2class = {v: k for k, v in test_dataset.class_to_idx.items()}
df_teste = pd.DataFrame.from_dict([get_class_distribution(test_dataset)]).melt()

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(
    labels=df_treino['variable'],
     values=df_treino['value'],
      pull=[0, 0, 0.1, 0]),
       1, 1)

fig.add_trace(go.Pie(
    labels=df_teste['variable'],
     values=df_teste['value'],
      pull=[0, 0, 0.1, 0]),
       1, 2)

fig.update_traces(hole=.4,
    hoverinfo='label+percent',
                  textinfo='value',
                  textfont_size=12.5,
                  marker=dict(
                      line=dict(
                          color='#000000',
                           width=1)
                      )
                  )

fig.update_layout(
    title_text = 'Proporção da base de dados de treino e teste',
    annotations=[dict(text='Treino', x=0.181, y=0.5, font_size=20, showarrow=False),
                 dict(text='Teste', x=0.80, y=0.5, font_size=20, showarrow=False)]
)

fig.show()

## Undersample

In [9]:
quantidade = os.listdir('/content/gdrive/MyDrive/MRI_TUMOR_CNN/80_20_Brain_Tumor/BrainTumor_80/no_tumor')
total_nao_tumores = len(quantidade)
print(total_nao_tumores)

400


In [10]:
path_glioma = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/80_20_Brain_Tumor/BrainTumor_80/glioma_tumor/'
print("Transferindo...\n")
files = os.listdir(path_glioma)

quantidade_de_arquivos = os.listdir(path_glioma)
total = len(quantidade_de_arquivos)
diferenca = total - total_nao_tumores

for file in sample(files, diferenca):
  shutil.move(path_glioma + file, '/content/gdrive/MyDrive/MRI_TUMOR_CNN/80_20_lixeira_dataset/Glioma_copia')
print("Arquivos transferidos")

Transferindo...

Arquivos transferidos


In [11]:
path_meningioma = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/80_20_Brain_Tumor/BrainTumor_80/meningioma_tumor/'
print("Transferindo...\n")
files = os.listdir(path_meningioma)

quantidade_de_arquivos = os.listdir(path_meningioma)
total = len(quantidade_de_arquivos)
diferenca = total - total_nao_tumores

for file in sample(files, diferenca):
  shutil.move(path_meningioma + file, '/content/gdrive/MyDrive/MRI_TUMOR_CNN/80_20_lixeira_dataset/Meningioma_copia')
print("Arquivos transferidos")

Transferindo...

Arquivos transferidos


In [12]:
path_pituitary = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/80_20_Brain_Tumor/BrainTumor_80/pituitary_tumor/'
print("Transferindo...\n")
files = os.listdir(path_pituitary)

quantidade_de_arquivos = os.listdir(path_pituitary)
total = len(quantidade_de_arquivos)
diferenca = total - total_nao_tumores

for file in sample(files, diferenca):
  shutil.move(path_pituitary + file, '/content/gdrive/MyDrive/MRI_TUMOR_CNN/80_20_lixeira_dataset/Hipofise_copia')
print("Arquivos transferidos")

Existe?....True


In [13]:
dir_path_train = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/80_20_Brain_Tumor/BrainTumor_80'
dir_path_test = '/content/gdrive/MyDrive/MRI_TUMOR_CNN/80_20_Brain_Tumor/BrainTumor_20'

In [14]:
train_dataset = datasets.ImageFolder(dir_path_train, transform=transform_file)
test_dataset = datasets.ImageFolder(dir_path_test, transform=transform_file)

In [15]:
idx2class = {v: k for k, v in train_dataset.class_to_idx.items()}
df_treino = pd.DataFrame.from_dict([get_class_distribution(train_dataset)]).melt()

idx2class = {v: k for k, v in test_dataset.class_to_idx.items()}
df_teste = pd.DataFrame.from_dict([get_class_distribution(test_dataset)]).melt()

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(
    labels=df_treino['variable'],
     values=df_treino['value'],
      pull=[0, 0, 0.1, 0]),
       1, 1)

fig.add_trace(go.Pie(
    labels=df_teste['variable'],
     values=df_teste['value'],
      pull=[0, 0, 0.1, 0]),
       1, 2)

fig.update_traces(hole=.4,
    hoverinfo='label+percent',
                  textinfo='value',
                  textfont_size=12.5,
                  marker=dict(
                      line=dict(
                          color='#000000',
                           width=1)
                      )
                  )

fig.update_layout(
    title_text = 'Proporção da base de dados de treino e teste',
    annotations=[dict(text='Treino', x=0.181, y=0.5, font_size=20, showarrow=False),
                 dict(text='Teste', x=0.80, y=0.5, font_size=20, showarrow=False)]
)

fig.show()